In [4]:
import pandas as pd
#input_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/gisaid_data/main_MSA_files/filtered_msaCodon_1024_trimmed_RBD_3mil_with_VOC.csv"
input_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/filtered_msaCodon_1024_trimmed_RBD_15mil.csv"
in_df=pd.read_csv(input_csv_path)

In [5]:
in_df.head()

,Accession ID,Lineage,RBD nucleotide,Variant,VOC
0,EPI_ISL_10839200,BA.1.15,ACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACT...,Former VOC Omicron GRA (B.1.1.529+BA.*),Omicron
1,EPI_ISL_5652980,P.1.11,ACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACT...,Former VOC Gamma GR/501Y.V3 (P.1+P.1.*),Gamma
2,EPI_ISL_17397934,XBB.1.9.2,ACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACT...,VUM GRA (XBB.1.9.2+XBB1.9.2.*),VUM_GRA
3,EPI_ISL_9443030,BA.1,ACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACT...,Former VOC Omicron GRA (B.1.1.529+BA.*),Omicron
4,EPI_ISL_4353158,AY.4,ACTTGACCCTCTCTCAGAAACAAAGTGTACGTTGAAATCCTTCACT...,Former VOC Delta GK (B.1.617.2+AY.*),Delta


In [8]:
voc_counts = in_df['VOC'].value_counts()
print (voc_counts)

VOC
Omicron    5450788
Delta      3888578
Alpha      1043401
nonVOC      940219
VUM_GRA     269553
Eris        167378
Gamma       113831
Epsilon      58365
Iota         37878
Beta         14825
Mu           11307
Lambda        9169
Eta           7079
Zeta          6188
FLiRT         4277
Kappa         3594
VUM_GH         805
VOI_GRA        207
Theta           85
Name: count, dtype: int64


In [3]:
# Path2) Use Variant column value from metadata to extract Variant label
# Creating uniform variant label values from trimmed RBD dataset and mapping labels to numeric values
import pandas as pd
import csv

def transform_csv(input_csv, output_csv):
    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Create the new 'label_name' column by prefixing 'sars_cov_2_' to the 'VOC' column
    df['label_name'] = 'sars_cov_2_' + df['VOC']

    # Remove rows where 'label_name' is 'sars_cov_2_nonVOC'
    df = df[df['label_name'] != 'sars_cov_2_nonVOC']
    
    # Removing VOI and VUM labels
    df = df[df['label_name'] != 'sars_cov_2_VUM_GRA']
    df = df[df['label_name'] != 'sars_cov_2_VUM_GH']
    df = df[df['label_name'] != 'sars_cov_2_VOI_GRA']
    df = df[df['label_name'] != 'sars_cov_2_Iota']
    df = df[df['label_name'] != 'sars_cov_2_Epsilon']
    

    # Create a mapping from unique 'label_name' to a numeric value
    label_mapping = {label: idx for idx, label in enumerate(df['label_name'].unique())}

    # Create the 'label_number' column by mapping 'label_name' to its corresponding numeric value
    df['label_number'] = df['label_name'].map(label_mapping)

    # Rename columns
    df = df.rename(columns={'Accession ID': 'EPI_ID', 'RBD nucleotide': 'sequence'})

    # Select relevant columns and write them to the output CSV file
    df[['EPI_ID', 'sequence', 'label_name', 'label_number']].to_csv(output_csv, index=False)

    # Print the mapping of label names to numbers
    print("Mapping of label names to numeric values:")
    for label, number in label_mapping.items():
        print(f"{label}: {number}")

if __name__ == "__main__":
    input_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/filtered_msaCodon_1024_trimmed_RBD_15mil.csv"
    output_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/RBD_nucleotides_15mil_wo_nonvoc.csv"
    transform_csv(input_csv_path, output_csv_path)


Mapping of label names to numeric values:
sars_cov_2_Omicron: 0
sars_cov_2_Gamma: 1
sars_cov_2_Delta: 2
sars_cov_2_Alpha: 3
sars_cov_2_Eris: 4
sars_cov_2_Lambda: 5
sars_cov_2_Zeta: 6
sars_cov_2_Kappa: 7
sars_cov_2_Beta: 8
sars_cov_2_Mu: 9
sars_cov_2_Eta: 10
sars_cov_2_FLiRT: 11
sars_cov_2_Theta: 12


In [11]:
# Checking for ambigious nucleotides
import pandas as pd
#rbd_df= pd.read_csv('/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/RBD_nucleotides_15mil_wo_nonvoc.csv')

# Look for presence of ambigious nucleotide N in any sequence
if rbd_df['sequence'].str.contains('N').any():
    print("There is at least one 'N' in the 'RBD nucleotide' column.")
else:
    print("No 'N' found in the 'RBD nucleotide' column.")

No 'N' found in the 'RBD nucleotide' column.


In [3]:
# Calculating the number of unique EPI IDs for each unique value of virus label_name
import os
import pandas as pd

csv_file_path = '/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/RBD_nucleotides_15mil_wo_nonvoc.csv'

# Load the CSV file
data = pd.read_csv(csv_file_path)

# Group by label_name and count unique EPI_ID values
label_counts = data.groupby('label_name')['EPI_ID'].nunique()

# Print out the counts
for label, count in label_counts.items():
    print(f"Label: {label}, Unique EPI_ID Count: {count}")
    
# Group by 'label_name' and count the number of sequences for each
label_name_sequence_counts = data['label_name'].value_counts()

# Print the unique label_name values and their corresponding sequence counts
#for label_name, count in label_name_sequence_counts.items():
    #print(f"label_name: {label_name}, sequence count: {count}")


Label: sars_cov_2_Alpha, Unique EPI_ID Count: 1043401
Label: sars_cov_2_Beta, Unique EPI_ID Count: 14825
Label: sars_cov_2_Delta, Unique EPI_ID Count: 3888578
Label: sars_cov_2_Eris, Unique EPI_ID Count: 167378
Label: sars_cov_2_Eta, Unique EPI_ID Count: 7079
Label: sars_cov_2_FLiRT, Unique EPI_ID Count: 4277
Label: sars_cov_2_Gamma, Unique EPI_ID Count: 113831
Label: sars_cov_2_Kappa, Unique EPI_ID Count: 3594
Label: sars_cov_2_Lambda, Unique EPI_ID Count: 9169
Label: sars_cov_2_Mu, Unique EPI_ID Count: 11307
Label: sars_cov_2_Omicron, Unique EPI_ID Count: 5450788
Label: sars_cov_2_Theta, Unique EPI_ID Count: 85
Label: sars_cov_2_Zeta, Unique EPI_ID Count: 6188


In [12]:
# Path1) Creating subset of above output csv with desired number of unique EPI_IDs for each hCOV19 variant label without complementary sequence
import pandas as pd
import csv
import os
from collections import defaultdict

def filter_and_write_csv(input_csv, output_csv, min_rows_per_variant=100000, max_rows_per_variant=100000):
    # Initialize a dictionary to count the number of valid rows per variant
    variant_counts = defaultdict(int)
    variant_valid_rows = defaultdict(list)

    with open(input_csv, 'r') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            sequence = row['sequence'].upper()

            # Check if the sequence contains only A, T, G, C
            if set(sequence) <= {'A', 'T', 'G', 'C'}:
                variant = row['label_name']
                variant_valid_rows[variant].append(row)

    # Filter out variants that do not have at least min_rows_per_variant valid rows
    valid_variants = {variant: rows for variant, rows in variant_valid_rows.items() if len(rows) >= min_rows_per_variant}

    # Write to output CSV
    with open(output_csv, 'w', newline='') as outfile:
        fieldnames = ['EPI_ID', 'label_name', 'label_number', 'sequence']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        total_rows = 0
        for variant, rows in valid_variants.items():
            # Write only max_rows_per_variant rows for each variant
            for row in rows[:max_rows_per_variant]:
                writer.writerow(row)
                variant_counts[variant] += 1
                total_rows += 1

    print(f"Total number of rows in the output CSV: {total_rows}")
    for variant, count in variant_counts.items():
        print(f"Variant: {variant}, Number of rows: {count}")

if __name__ == "__main__":
    input_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/RBD_nucleotides_15mil_wo_nonvoc.csv"
    
    main_dir = "/mmfs1/projects/changhui.yan/DeewanB/DNABert2_rnaseq/genome_files/unfiltered_multiple_genomes/"

    output_csv_path = os.path.join(main_dir, "RBD_whole_nucleotides_15mil_wo_nonvoc_100k_epi.csv")
                                   
    filter_and_write_csv(input_csv_path, output_csv_path)


Total number of rows in the output CSV: 500000
Variant: sars_cov_2_Omicron, Number of rows: 100000
Variant: sars_cov_2_Gamma, Number of rows: 100000
Variant: sars_cov_2_Delta, Number of rows: 100000
Variant: sars_cov_2_Alpha, Number of rows: 100000
Variant: sars_cov_2_Eris, Number of rows: 100000


In [1]:
# Path2) Creating subset of above output csv with desired number of unique EPI_IDs for each hCOV19 variant label with complementary sequence

import pandas as pd
import csv
import os
from collections import defaultdict
from Bio.Seq import Seq  # Import Biopython's Seq class

# Function to compute the reverse complement of a sequence using Biopython
def reverse_complement(sequence):
    return str(Seq(sequence).reverse_complement())  # Biopython's reverse complement

def filter_and_write_csv(input_csv, output_csv, min_rows_per_variant=100000, max_rows_per_variant=100000):
    # Initialize a dictionary to count the number of valid rows per variant
    variant_counts = defaultdict(int)
    variant_valid_rows = defaultdict(list)

    with open(input_csv, 'r') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            sequence = row['sequence'].upper()

            # Check if the sequence contains only A, T, G, C
            if set(sequence) <= {'A', 'T', 'G', 'C'}:
                variant = row['label_name']
                variant_valid_rows[variant].append(row)

    # Filter out variants that do not have at least min_rows_per_variant valid rows
    valid_variants = {variant: rows for variant, rows in variant_valid_rows.items() if len(rows) >= min_rows_per_variant}

    # Write to output CSV
    with open(output_csv, 'w', newline='') as outfile:
        fieldnames = ['EPI_ID', 'label_name', 'label_number', 'sequence']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        total_rows = 0
        for variant, rows in valid_variants.items():
            # Write only max_rows_per_variant rows for each variant
            for row in rows[:max_rows_per_variant]:
                original_sequence = row['sequence']
                rev_complement_sequence = reverse_complement(original_sequence)

                # Write original row
                writer.writerow(row)

                # Write reverse complement row
                reverse_complement_row = row.copy()
                reverse_complement_row['sequence'] = rev_complement_sequence
                writer.writerow(reverse_complement_row)

                variant_counts[variant] += 2  # Counting both original and reverse complement rows
                total_rows += 2

    print(f"Total number of rows in the output CSV: {total_rows}")
    for variant, count in variant_counts.items():
        print(f"Variant: {variant}, Number of rows: {count}")

if __name__ == "__main__":
    input_csv_path = "/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/RBD_nucleotides_15mil_wo_nonvoc.csv"
    
    main_dir = "/mmfs1/projects/changhui.yan/DeewanB/DNABert2_rnaseq/genome_files/unfiltered_multiple_genomes/"

    output_csv_path = os.path.join(main_dir, "RBD_whole_nucleotides_15mil_wo_nonvoc_100k_epi_wt_comp.csv")
                                   
    filter_and_write_csv(input_csv_path, output_csv_path)

Total number of rows in the output CSV: 1000000
Variant: sars_cov_2_Omicron, Number of rows: 200000
Variant: sars_cov_2_Gamma, Number of rows: 200000
Variant: sars_cov_2_Delta, Number of rows: 200000
Variant: sars_cov_2_Alpha, Number of rows: 200000
Variant: sars_cov_2_Eris, Number of rows: 200000


In [2]:
# Counting sequence statistics for trimmed RBD fasta file to find out if outliers exist

import pandas as pd

def generate_sequence_statistics_from_csv(csv_file, sequence_column):
    sequence_lengths = []
    total_sequences = 0
    total_bases = 0

    # Load the CSV file and extract the sequences from the specified column
    df = pd.read_csv(csv_file)

    # Iterate over each sequence in the column
    for sequence in df[sequence_column]:
        total_sequences += 1
        sequence_length = len(sequence)
        sequence_lengths.append(sequence_length)
        total_bases += sequence_length

    # Calculate statistics
    min_length = min(sequence_lengths)
    max_length = max(sequence_lengths)
    average_length = total_bases / total_sequences

    # Print the statistics
    print("Sequence Statistics:")
    print(f"Total Sequences: {total_sequences}")
    print(f"Total Bases: {total_bases}")
    print(f"Minimum Length: {min_length} bases")
    print(f"Maximum Length: {max_length} bases")
    print(f"Average Length: {average_length:.2f} bases")

if __name__ == "__main__":
    csv_file = "/mmfs1/projects/changhui.yan/DeewanB/DNABert2_rnaseq/genome_files/unfiltered_multiple_genomes/RBD_whole_nucleotides_15mil_wo_nonvoc_100k_epi_wt_comp.csv"  # Replace with your CSV file path
    sequence_column = "sequence"  # Replace with the column name that contains the sequences
    generate_sequence_statistics_from_csv(csv_file, sequence_column)


Sequence Statistics:
Total Sequences: 1000000
Total Bases: 633000000
Minimum Length: 633 bases
Maximum Length: 633 bases
Average Length: 633.00 bases


In [3]:
# Generating counts for each variant label and total number of RBD sequences
# Checking to see if label_names are correctly mapped to label_numbers

import pandas as pd

# Load the data from CSV into a DataFrame
rbd_df = pd.read_csv('/mmfs1/projects/changhui.yan/DeewanB/DNABert2_rnaseq/genome_files/unfiltered_multiple_genomes/RBD_whole_nucleotides_15mil_wo_nonvoc_100k_epi_wt_comp.csv')

# Function to check label numbers, count unique EPI_IDs, and count fragments for each label_name
def check_label_numbers_and_count_epi_ids(df):
    """
    Check the label_number values for each unique label_name value, print the number of EPI_ID values for each label_name,
    and also print the number of fragments (sequences) for each label_name.
    """
    # Group by label_name and analyze label_number, EPI_ID counts, and fragment counts
    grouped = df.groupby('label_name')

    for label_name, group in grouped:
        unique_label_numbers = group['label_number'].unique()
        epi_id_count = group['EPI_ID'].nunique()  # Count unique EPI_ID values
        fragment_count = group.shape[0]  # Count total number of fragments (sequences)

        print(f"Label Name: {label_name}")
        print(f"  - Unique Label Numbers: {unique_label_numbers}")
        print(f"  - Number of Unique EPI_IDs: {epi_id_count}")
        print(f"  - Number of Fragments (Sequences): {fragment_count}")
        print("-" * 40)

# Run the function on the DataFrame
check_label_numbers_and_count_epi_ids(rbd_df)



Label Name: sars_cov_2_Alpha
  - Unique Label Numbers: [3]
  - Number of Unique EPI_IDs: 100000
  - Number of Fragments (Sequences): 200000
----------------------------------------
Label Name: sars_cov_2_Delta
  - Unique Label Numbers: [2]
  - Number of Unique EPI_IDs: 100000
  - Number of Fragments (Sequences): 200000
----------------------------------------
Label Name: sars_cov_2_Eris
  - Unique Label Numbers: [4]
  - Number of Unique EPI_IDs: 100000
  - Number of Fragments (Sequences): 200000
----------------------------------------
Label Name: sars_cov_2_Gamma
  - Unique Label Numbers: [1]
  - Number of Unique EPI_IDs: 100000
  - Number of Fragments (Sequences): 200000
----------------------------------------
Label Name: sars_cov_2_Omicron
  - Unique Label Numbers: [0]
  - Number of Unique EPI_IDs: 100000
  - Number of Fragments (Sequences): 200000
----------------------------------------


In [4]:
#PATH3) Creating each one of 50, 75, 100, 150 and 250bps fragments with 50 bp overlaps from WGS generated above
# Generating reverse complementary sequence for each generated fragment 

import os
import pandas as pd
import csv
from tqdm import tqdm
from Bio.Seq import Seq

# Function to generate overlapping fragments of varying lengths
def generate_overlapping_fragments(sequence, fragment_len, overlap_ratio=0.2):
    overlap = int(fragment_len * overlap_ratio)  # Calculate overlap as 20% of fragment length
    fragments = []
    for start in range(0, len(sequence) - fragment_len + 1, fragment_len - overlap):
        fragment = sequence[start:start + fragment_len]
        if len(fragment) == fragment_len:
            fragments.append(fragment)
    return fragments

# Prompt user for the main directory path
input_dir = "/mmfs1/projects/changhui.yan/DeewanB/recent_hcov19_MSA/msaCodon_0522/"
output_dir = "/mmfs1/projects/changhui.yan/DeewanB/DNABert2_rnaseq/genome_files/unfiltered_multiple_genomes/"

# Path to the input and output CSV files
input_csv_path = os.path.join(input_dir, "RBD_nucleotides_15mil_wo_nonvoc_100k_epi.csv")
output_csv_path = os.path.join(output_dir, "RBD_nucleotides_15mil_wo_nonvoc_20percent_overlaps_complementary_100k_epi.csv")

# List of fragment lengths to generate
fragment_lengths = [50, 150, 250]

# Read the input CSV file in chunks
chunksize = 1000  # Adjust the chunk size as needed
input_columns = ["EPI_ID", "label_name", "label_number", "sequence"]

# Open the output CSV file for writing
with open(output_csv_path, mode='w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["EPI_ID", "label_name", "label_number", "fragment_length", "sequence"])  # Write header
    
    # Process the input CSV file in chunks
    for chunk in tqdm(pd.read_csv(input_csv_path, usecols=input_columns, chunksize=chunksize)):
        for index, row in chunk.iterrows():
            epi_id = row["EPI_ID"]
            variant_label = row["label_name"]
            variant_label_number = row["label_number"]
            sequence = row["sequence"]
            
            # Generate and write fragments for each fragment length
            for fragment_len in fragment_lengths:
                fragments = generate_overlapping_fragments(sequence, fragment_len)
                
                for fragment in fragments:
                    # Write the original fragment with the fragment length
                    writer.writerow([epi_id, variant_label, variant_label_number, fragment_len, fragment])
                    
                    # Generate and write the reverse complementary fragment
                    reverse_complement = str(Seq(fragment).reverse_complement())
                    writer.writerow([epi_id, variant_label, variant_label_number, fragment_len, reverse_complement])

print(f"Fragments with reverse complements for all fragment lengths have been written to {output_csv_path}")

500it [01:51,  4.50it/s]

Fragments with reverse complements for all fragment lengths have been written to /mmfs1/projects/changhui.yan/DeewanB/DNABert2_rnaseq/genome_files/unfiltered_multiple_genomes/RBD_nucleotides_15mil_wo_nonvoc_20percent_overlaps_complementary_100k_epi.csv
